<a href="https://colab.research.google.com/github/sgr1118/deep-learning-from-scratch-3/blob/main/%EC%A0%9C2%EA%B3%A0%EC%A7%80_%EC%9E%90%EC%97%B0%EC%8A%A4%EB%9F%AC%EC%9A%B4_%EC%BD%94%EB%93%9C%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표
- 더 복잡한 계산을 수행하는 DeZero 확장
- DeZero 파이썬 패키지로 묶어주기

In [ ]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3/'

/content/drive/MyDrive/Book/deeplearng_basic_3


## 11단계 : 가변 길이 인수(순전파 편)
- 함수는 한 개의 변수가 아닌 여러개의 변수를 입력받기도 한다.
- 가변 길이는 인수(또는 반환값)의 수가 달라질 수 있다는 것

### 11.1 Function 클래스 수정
- 가변 길이 입출력을 표현하려면 변수들을 리스트 또는 튜플을 넣어 처리하는 것이 편하다.
- __call__ 메서드의 인수와 반환값을 리스트로 바꿔본다.

In [ ]:
# 추가되어진 코드
class Funtion:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

### 11.2 Add 클래스 구현
- Add 클래스를 구현한다. 다만 인수와 반환값이 리스트 또는 튜플이여야한다.
- 아래 코드는 더 자연스럽게 개선이 필요하다.

In [ ]:
# 추가 코드
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 튜플 형태 반환값, 값이 하나 생략되어도 문제 없다.

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

In [ ]:
!python step11.py

5


## 12단계 : 가변 길이 인수(개선 편)
- Add 클래스를 이용하는 사람을 위한 개선
- 구현하는 사람을 위한 개선

### 12.1 첫 번째 개선 : 함수를 사용하기 쉽게
- 기존 클래스는 인수를 리스트에 모아서 전달받지만 개선하면 이 같은 과정을 거치지 않아도된다.

In [ ]:
# 추가 코드
class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
!python step12.py

5


### 12.2 두 번째 개선: 함수를 구현하기 쉽도록
- 기존 Add 클래스는 인수는 리스트로 전달되고 결과는 튜플로 반환했다.
- 개선하면 입력, 변수를 직접 받고 결과도 변수를 직접 돌려준다.

In [ ]:
# 추가 코드
class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
!python step12.py

5


### 12.3 add 함수 구현
- 마지막으로 Add 클래스를 '파이썬 함수'로 사용할 수 있는 코드 추가

In [ ]:
# 추가 코드
def add(x0, x1):
    return Add()(x0, x1)

In [ ]:
!python step12.py

5


## 13단계 : 가변 길이 인수(역전파 편)

### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파
- 상류에서 온 미분값을 '그대로 흘려보내는 것'이 덧셈의 역전파이다.

In [ ]:
# 추가 코드
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy): # 입력이 1개, 출력이 2개
        return gy, gy

### 13.2 Variable 클래스 수정
- 앞서 Variable 클래스는 함수의 입출력이 하나라고 가정했으나 이 부분을 여러 개의 변수가 대응할 수 있도록 수정

In [ ]:
# 추가 코드
def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

### 13.3 Square 클래스 구현
- Square 클래스또한 새로운 Variable과 Function에 맞게 수정할 필요가 있다.

In [ ]:
# 추가 코드

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 복수형 inputs로 변경
        gx = 2 * x * gy
        return gx

    def squqre(x):
        f = Square()
        return f(x)

In [ ]:
!python step13.py

13.0
4.0
6.0


- $z = x^2 + y^2$이라는 계산을 z = add(square(x), square(y))라는 코드로 풀어냈다. 그런 다음 z.backward()를 호출하면 미분 계산이 자동으로 이루어진다.

## 14단계: 같은 변수 반복 사용
- 현재까지 코드에서 같은 변수를 반복 사용하면 의도대로 동작하지 않을 수 있다는 문제가 있다.
- 동일한 변수를 사용하면 덧셈을 제대로 미분할 수 없다.

### 14.1 문제의 원인
- step13.py에서 Variable 클래스에 문제가 있다. 현재는 전해지는 미분값을 그대로 대입하고 있다. 즉 같은 변수를 반복해서 사용하면 미분값이 덮어 씌여진다.
- 해결하기 위해 전파되는 미분값의 '합'이 필요하다.

### 14.2 해결책
- 미분값의 '합'을 구하는 코드 구현

In [ ]:
# 코드 수정

class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # 메서드 추가
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx 

                if x.creator is not None:
                    funcs.append(x.creator)

In [ ]:
!python step14.py

2.0
3.0


### 14.3 미분값 재설정
- 문제가 해결된 듯 보이지만 또 다른 문제가 있다. 같은 변수를 사용하여 '다른' 계산을 할 경우 계산이 꼬이는 문제이다.
- 인스턴스를 재사용하는 바람에 두 번째 계산 값에 첫 번째 미분 값이 더해지고 말았다.
- 해결하기 위해 Variable 클래스에 미분값을 초기화하는 cleargead 메서드 추가

In [ ]:
!python step14.py

2.0
5.0


In [ ]:
# 추가 코드
def cleargrad(self):
    self.grad = None # 여러 가지 미분을 계산할 때 똑같은 변수를 재사용 가능

In [ ]:
!python step14.py

2.0
3.0


## 15단계: 복잡한 계산 그래프(이론 편)
- 현재 우리가 구현한 Dezero는 일직선 계산 그래프를 잘 수행하지만 더 복잡한 계산 그래프의 역전파를 제대로 할 수 없다.

### 15.1 역전파의 올바른 순서

![](https://velog.velcdn.com/images/hyxxnii/post/3e7d8eb2-4eff-442b-84e4-6b8bb79d980c/image.png)
<center>중간에 분기했다가 다시 합류하는 계산 그래프</center>

- 현재 DeZero는 위와 같은 계산 그래프의 역전파도 수행할 수 없다.
- 계산 중간에 'a'라는 변수는 역전파 출력시 미분값이 더해야 한다. 'a'의 미분을 계산하려면 'a'의 출력 쪽에서 전파하는 2개의 미분값이 필요하다.
- 그 2개의 미분값이 전달된 후에야 a > x 전파가 가능하다.

### 15.2 현재의 DeZero
- Variable 클래스에서 funcs 리스트를 자세히봐라.
- while의 마지막 code에서 처리할 함수의 후보를 funcs 리스트의 끝에 추가하고 있다. 그 다음 처리할 함수를 그 리스트의 끝에서 꺼낸다.
- 이렇게 된다면 함수의 처리 순서가 D,C,A,B,A가 되어 버린다.
- 역전파 과정을 다시 생각해보자. funcs 리스트에 D가 추가되어 [D] 상태로 시작된다. 그 다음 D의 입력 변수인 B와 C가 리스트에 추가될 것이다.
- 이 시점에서 funcs 리스트는 [B,C]이다. 그 다음 마지막 원소 C를 꺼내고 A가 리스트에 추가되어 [B,A]가 된다. 그럼 다시 마지막 원소인 A를 꺼내버린다. 원래 B가 나왔어야했다.

In [ ]:
class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # 메서드 추가
        self.creator = func

    def cleargrad(self):
        self.grad = None # 여러 가지 미분을 계산할 때 똑같은 변수를 재사용 가능

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 미분값을 더해준다.

                if x.creator is not None:
                    funcs.append(x.creator)

### 15.3 함수 우선순위
- 이런 문제가 발생한 이유는 함수 우선순위를 설정하지 않았기 때문이다. 우선순위를 설정하는 가장 쉬운 방법은 함수와 변수 간 관계를 활용하는 것이다. 이 관계를 기준으로 함수와 변수의 '세대'를 기록하고 이것이 우선순위인 것이다.

## 16단계: 복잡한 계산 그래프(구현 편)

### 16.1 세대 추가
- Variable, Function 클래스에 인스턴스 변수 generation을 추가한다.

In [ ]:
# 추가 코드

class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가
        self.generation = 0 # 세대 수를 기록하는 변수

    def set_creator(self, func): # 메서드 추가
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록 (부모 세대 + 1)

In [ ]:
# 추가 코드

class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        # 입력 변수와 같은 값으로 generation 설정
        self.generation = max([x.generation for x in inputs])
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

### 16.3 Variable 클래스의 backward
- add_func 함수를 backward 메서드 안에 중첩 함수로 정의했다. 중첩 함수는 주로 다음 두 조건을 충족할 때 적합하다.
- 감싸는 메서드(backward 메서드) 안에서만 이용한다.
- 감싸는 메서드(backward 메서드)에 정의된 변수(funcs, seen_set)를 사용해야 한다.

In [ ]:
# 추가 코드
class Variable:

   def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성

        funcs = []
        seen_set = set() # 함수의 backward 메서드가 여러 번 불리는 것 방지

        def add_func(f): # 함수 리스트 세대 순으로 정렬
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 자동으로 세대가 가장 큰 DeZero 함수를 꺼낸다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 미분값을 더해준다.

                if x.creator is not None:
                    add_func(x.creator) 

### 16.4 동작 확인

In [ ]:
!python step16.py

32.0
64.0


## 17단계: 메모리 관리와 순환 참조
- 성능을 개선할 수 있는 대책을 적용할 예정이다.

### 17.1 메모리 관리
- 파이썬은 필요없어진 객체를 삭제한다. 그렇지만 코드를 제대로 작성하지 않으면 메모리 부족 등의 문제가 발생한다. 특히 신경망에서는 대량의 데이터를 다루기에 메모리 관리를 제대로 하지않으면 실행 시간이 매우 오래걸린다.
- 파이썬 메모리 관리 방식은 두 가지이다. 첫 번째는 참조 수를 세는 방식, 두 번째는 세대를 기준으로 쓸모 없어진 객체를 회수하는 방식이다.

### 17.2 참조 카운트 방식의 메모리 관리
- 파이썬 메모리 관리의 기본은 참조 카운트 방식이다. 객체가 참조되어지면 +1, 필요 없어지면 -1만큼 감소하여 0이되면 인터프리터가 회수된다. 아래와 같은 경우 참조 카운트가 증가한다.

- 대입 연산자를 사용
- 함수에 인주로 전당할 때
- 컨테이너 타입 객체(리스트, 튜플, 클래스 등)을 추가할 때

- 하지만 이런 방법으로도 해결할 수 없는 문제가 바로 순환 참조이다.

In [ ]:
# 참조 카운트 예시

class obj:
    pass

a = obj()
b = obj()
c = obj()

a, b = b
b, c = c

a = b = c = None # a가 참조 카운트가 0이 되고 그 다음 b, c순으로 0이 된다. 

### 17.3 순환 참조
- 아래 코드처럼 서로가 서로를 참조하는 것을 순환 참조라고한다. 이런 경우 참조 카운트가 0이 되지 않기 때문에 GC('세대별 가비지 컬렉션')방법을 사용한다.
- GC는 불필요한 객체를 찾아내어 메모리 부족 문제를 해소한다.

In [ ]:
# 순환 참조

class obj:
    pass

a = obj()
b = obj()
c = obj()

a, b = b
b, c = c
c, a = a 

a = b = c = None # 

### 17.4 weakref 모듈
- weakref.ref를 사용하여 약한 참조를 만들 수 있다. 이 기능은 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 것이다.

In [ ]:
# 추가 코드
import weakref

class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        # 입력 변수와 같은 값으로 generation 설정
        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] # 약한 참조

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

class Varuable:
    
    while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output().grad for output in f.outputs] # 약한 참조를 적용하고 출력 변수(미분값)을 리스트에 담는다.

### 17.5 동작 확인
- 순환 참조가 없어진 상태를 테스트한다.

In [ ]:
# 추가 코드

for i in range(10):
    x = Variable(np.random.randn(10000))  # 많은 데이터
    y = square(square(square(x))) # 복잡한 계산 수행

# for문이 두 번째 반복될 때 x, y가 덮어 씌어져, 이전의 계산 그래프를 참고하지 않게 된다.

In [ ]:
!python step17.py

## 18단계: 메모리 절약 모드
- 역전파 시 사용하는 메모리를 줄이는 방법, 불필요한 미분 결과를 보관하지 않고 삭제
- '역전파가 필요 없는 경우'모드 제작, 불필요한 계산을 생략한다.

### 18.1 필요 없는 미분값 삭제
- x0과 x1은 사용자가 제공하는 변수이기에 미분값이 필요하지만, y, t같은 중간 변수는 필요하지 않다. 이 미분값을 제거해주도록 한다.

In [ ]:
!python step18.py

1.0 1.0
2.0 1.0


In [ ]:
# 추가 코드

class Variable():

    def backward(self, retain_grad = False): # retain_grad = False는 중간 변수 미분값을 모두 None으로 설정
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성

        funcs = []
        seen_set = set() # 함수의 backward 메서드가 여러 번 불리는 것 방지

        def add_func(f): # 함수 리스트 세대 순으로 정렬
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)
        
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output().grad for output in f.outputs] # 약한 참조를 적용하고 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 미분값을 더해준다.

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None # y는 약한 참조, 각 함수의 출력 변수의 미분값을 
                    # 유지하지 않도록 설정

In [ ]:
!python step18.py
# y, t의 미분 값이 삭제된 것을 확인하였다.

None None
2.0 1.0


### 18.2 Function 클래스 복습
- self.inputs는 순전파 때 결과값을 기억해준다. 
- 인스턴스 변수인 inputs를 참조하지 않았다면 참조 카운트가 0이 되어 메모리에서 삭제됐을 것이다.

In [ ]:
class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        # 입력 변수와 같은 값으로 generation 설정
        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs # 순전파 때 결과값 기억
        self.outputs = [weakref.ref(output) for output in outputs] # 약한 참조

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

### 18.3 config 클래스를 활용한 모드 전환
- 역전파 활성화/비활성화 모드를 제작하도록한다.

In [ ]:
# 추가 코드

class Config:
    enable_backprop = True # 역전파 가능 여부, 'True' : 역전파 활성화 모드

# 추가 코드 - 2

class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        if Config.enable_backprop:

            # 입력 변수와 같은 값으로 generation 설정
            # '역전파 비활성화 시' 세대 설정은 필요하지 않다. 
            self.generation = max([x.generation for x in inputs]) # 세대 설정

            for output in outputs:
                output.set_creator(self) # 연결 설정
            self.inputs = inputs # 순전파 때 결과값 기억
            self.outputs = [weakref.ref(output) for output in outputs] # 약한 참조

            # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
            return outputs if len(outputs) > 1 else outputs[0]

### 18.4 모드 전환


In [ ]:
# 추가 코드 (테스트용)

Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.bacward()

In [ ]:
!python step18.py

None None
2.0 1.0


### 18.5 with 문을 활용한 모드 전환
- with는 후처리를 자동으로 수행할 때 사용하는 구문이다.
- 

In [ ]:
# 추가 코드

with using_config('enable_backprop', False): # '역전파 비활성 모드'
    x = Variable(np.array(2.0))
    y = square(x)

# 추가 코드 - 2
import contextlib

@contextlib.contextmanager
def using_config(name, value): # name : 사용할 Config 속성의 이름
    old_value = getattr(Config, name) # name을 getattr 함수에 넘겨 Config 클래스에서 꺼내온다.
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

# 추가 코드 - 3
def no_grad(): # 편의 함수
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

In [ ]:
!python step18.py

None None
2.0 1.0


## 19단계: 변수 사용성 개선
- DeZero를 더 쉽게 사용하기 위해 Variable 클래스를 조정하도록 한다.

### 19.1 변수 이름 지정
- 변수들은 서로 구분할 필요가 있다. Variable 클래스에 name이라는 인스턴스 변수 추가

In [ ]:
# 추가 코드

class Variable():
    def __init__(self, data, name = None): # 인스턴스 변수
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.name = name
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가
        self.generation = 0 # 세대 수를 기록하는 변수

### 19,2 ndarray 인스턴스 변수
- Variable 클래스는 데이터를 담는 상자 역할이다. 하지만 더 중요한 것은 그 안의 데이터이다. Variable이 데이터인 것처럼 보이게 하는 장치를 추가한다.

In [ ]:
# 추가 코드
class Variable:

    @property # shape 메서드를 인스턴스 변수처럼 사용하게한다.
    def shape(self):
        return self.data.shape # 인스턴스 변수로 데이터의 형상을 얻을 수 있다.

    @property
    def ndim(self): # 차원 수
        return self.data.ndim 

    @property 
    def size(self): # 원소 수 
        return self.data.size

    @property 
    def dtype(self): # 데이터 타입
        return self.data.dtype

### 19.3 len 함수와 print 함수
- len을 사용하여 객체 수를 알려주는 기능을 추가

In [ ]:
# 추가 코드
class Variable:

    def __len__(self): # 객체 수 알림
        return len(self.data) 

    def __repr__(self): # ndarray에 담긴 내용 출력
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9) # 줄을 바꾸고 새로운 줄 앞에 공백 9개를 삽입
        return 'variable(' + p + ')'

## 20단계: 연산자 오버로드(1)
- Variable 인스턴스를 ndarray 인스턴트처럼 보이게 만드는 것이 목표이다.
- +, * 연산자를 지원하도록 Variable을 확장할 것이다.

### 20.1 Mul 클래스 구현
- '곱셈'을 할 수 있게 클래스를 구현한다.
- 좀 더 편하게 연산자를 사용할 수 있도록 연산자 오버로드를 이용하자.

In [ ]:
# 추가 코드
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x0, gy * x1

def mul(x0, x1):
    return Mul()(x0, x1)

In [ ]:
!python step20.py

variable(7.0)
3.0
2.0


### 20.2 연산자 오버로드


In [ ]:
# 추가 코드

Variable.__mul__ = mul # 함수 자체에 할당시킴
Variable.__add__ = add

In [ ]:
!python step20.py

variable(7.0)
3.0
2.0


## 21단계: 연산자 오버로드(2)
- 아직 인스턴스와 np.array(2.0)처럼 ndarray 인스턴스는 함께 사용할 수 없다.
- Variable 인스턴스와 ndarray 인스턴스, 심지어 int, float 등도 함께 사용할 수 있게 한다.

### 21.1 ndarray와 함께 사용하기
- Variable과 ndarray 인스턴스를 함께 사용하기하려면 ndarray 인스턴스를 자동으로 Variable인스턴스로 변환시키면된다.

In [ ]:
# 추가 코드
def as_variable(obj):
    if isinstance(obj, Variable): # obj가 Variable 인스턴스 인지 확인
        return obj
    return Variable(obj) # 만약 Variable 인스턴스가 아니라면 변환

# 추가 코드 - 2

class Function:
    def __cal__(self, *inputs):
        inputs = [as_variable(x) for x in inputs] # inputs에 담긴 원소를 Variable로 변환

In [ ]:
!python step21.py

variable(5.0)


### 21.2 float, int와 함께 사용하기

In [ ]:
# 추가 코드
def add(x0, x1):
    x1 = as_array(x1) # ndarray 인스턴스로 변환
    return Add(x0, x1)

In [ ]:
!python step21.py

variable(5.0)


### 21.3 문제점 1: 첫 번째 인수가 float, int인 경우
- *과 같은 이랑 연산자의 경우 피연산자(항)의 위치에 따라 호출되는 특수 메서드가 다르다. 곱셈의 경우 피연산자가 좌항이면 __mul__, 우항이면 __rmul__메서드가 호출
- 이런 문제는 __rmul__ 메서드를 구현하면 해결

In [ ]:
# 추가 코드

Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [ ]:
!python step21.py

variable(7.0)


### 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우
- '연산자 우선순위'를 지정하여 해결 가능

In [ ]:
# 추가 코드
class Variable:
    __array_priority__ = 200 # 연산자 우선순위 지정


In [ ]:
!python step21.py

variable(7.0)


## 22단계: 연산자 오버로드(3)
- 더 많은 연산자들을 추가 지원하도록 한다.
- 작업은 아래 순서에 따라 진행한다.
- 1. Function 클래스를 상속하여 원하는 함수 클래스를 구현
- 2. 파이썬 함수로 사용할 수 있도록 한다.
- 3. Variable 클래스의 연산자를 오버로드한다.

### 22.1 음수(부호 변환)

In [ ]:
# 추가 코드
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [ ]:
!python step22.py

variable(-2.0)


### 22.2 뺄셈

In [ ]:
# 추가 코드
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0) # x0와 x1의 순서를 바꿔준다.

Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [ ]:
!python step22.py

variable(0.0)
variable(1.0)


### 22.3 나눗셈

In [ ]:
# 추가 코드

class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

    def div(x0, x1):
        x1 = as_array(x1)
        return Div()(x0, x1)

    def rdiv(x0, x1):
        x1 = as_array(x1)
        return div()(x1, x0) # x0과 x1의 순서를 바꾼다.

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [ ]:
!python step22.py

variable(1.5)


### 22.4 거듭제곱

In [ ]:
# 추가 코드

class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c-1) * gy
        return gx

def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

In [ ]:
!python step22.py

variable(8.0)


## 23단계: 패키지로 정리
- 지금까지 step을 패키지로 정리한다.
- 모듈 : 파이썬 파일, import하여 사용하는 것으 가정하고 만들어진 파이썬 파일
- 패키지 : 여러 모듈을 묶은 것, 패키지를 만들기 위해서는 디렉터리를 만들고 그 안에 모듈을 추가
- 라이브러리 : 여러 패키지를 묶은 것, 하나 이상의 디렉터리로 구성괸다.

### 23.1 파일 구성
- dezero라는 디렉터리에 모듈을 추가한다. 그럼 dezero라는 패키지가 만들어지고 이 것을 프레임워크라고 한다.

### 23.2 코어 클래스 옮기기

In [ ]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3/'

/content/drive/MyDrive/Book/deeplearng_basic_3


In [ ]:
import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
print(x)

variable(1.0)


### 23.3 연산자 오버로드
- setop_variable() 함수 호출을 위하여 __init__.py를 만든다.
- __init__.py는 모듈을 import할 때 가장 먼저 실행되는 파일이다.

In [ ]:
# 추가 코드

def setup_variable():
    Variable.__add__ = add
    Variable.__radd__ = add
    Variable.__mul__ = mul
    Variable.__rmul__ = mul
    Variable.__neg__ = neg
    Variable.__sub__ = sub
    Variable.__rsub__ = rsub
    Variable.__truediv__ = div
    Variable.__rtruediv__ = rdiv
    Variable.__pow__ = pow

# __init__.py code

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable

    setup_variable() # 함수를 import하여 호출
    # dezero 패키지에서 Variable 클래스를 곧바로 import 할 수 있다

In [ ]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3'

/content/drive/MyDrive/Book/deeplearng_basic_3


In [ ]:
# from dezero.core_simple import Variable
from dezero import Variable

### 23.4 실제 __init__.py 파일
- __init__.py에서 is_simple_core 플래그로 임포트할 대상을 선택한다. True면 core_simple.py에서 False면 core.py에서 임포트가 이루어진다.

### 23.5 dezero 임포트

In [ ]:
# 추가 코드
if '__file__' is globals(): # __file__이라는 전역 변수가 정의되어 있는지 확인
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from dezero import Variable

x = Variable(np.arrau(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

In [ ]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3/'

/content/drive/MyDrive/Book/deeplearng_basic_3


In [ ]:
!python steps/step23.py

variable(16.0)
8.0


## 24단계: 복잡한 함수의 미분
- 이 단계에서는 다루는 함수들은 최적화 문제에서 자주 사용되는 테스트 함수이다. 최적화 문제의 테스트 함수란 다양한 최적화 기법이 '얼마나 좋은가'를 평가하는데 사용되는 함수를 뜻한다.

### 24.1 Sphere 함수
- Sphere 함수는 두 개의 입력 변수를 제곱하여 더하는 함수이다.

In [ ]:
# 추가 코드
import numpy as np
from dezero import Variable

def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

In [ ]:
!python step24.py

/content/drive/MyDrive/Book/deeplearng_basic_3/step24.py:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if '__file__' is globals():
2.0 2.0


### 24.2 matyes 함수

In [ ]:
# 추가 코드
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

In [ ]:
!python step24.py

/content/drive/MyDrive/Book/deeplearng_basic_3/step24.py:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if '__file__' is globals():
3.52 3.0


### 24.3 Goldstein-Price 함수

In [ ]:
# 추가 코드
def goldstein(x, y):
     z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z 

In [ ]:
!python steps/step24.py

-84799.0 6772.0


- 제2고지에서는 복잡한 계산까지 가능하게되었다. 또한 연산자를 오버로드 한 덕에 보통의 파이썬 프로그래밍처럼 코드를 작성할 수 있다.